In [1]:
# Date: Dec 2, 2022
# Author: Sashka Warner
# Desc: Clean UWWI client dataset for use in calculating distances to services

# Import libraries
import pandas as pd

In [2]:
# Load client data from csv
clients_raw = pd.read_csv("../../uwwi_datasets/uwwi_dataset_clients_v2.csv")

/var/folders/bx/8_p36bpd14nckr66rq2kk6kc0000gn/T/ipykernel_18142/24448644.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  clients_raw = pd.read_csv("../../uwwi_datasets/uwwi_dataset_clients_v2.csv")


In [3]:
# Inspect the data
clients_raw.head()

,Client_Id,Client_CreateStamp,Client_EditStamp,ClientOption_PreferredLanguageOfCaller,ClientSystem_PreferredLanguageOther,ClientAddressus_ClientAddressus_city,ClientAddressus_ClientAddressus_county,ClientAddressus_ClientAddressus_state,ClientAddressus_ClientAddressus_zip,ClientCustom_AgeOfPersonNeedingAssistance,ClientCustom_EthnicityOther,ClientOption_GenderOptionId,ClientOption_VeteranStatusOptionlist
0,1,2017-04-07T15:34:04.872169-05:00,2022-04-23T00:53:31.812322-05:00,[],NaN,FOND DU LAC,FOND DU LAC,WI,54935.0,NaN,NaN,[],[]
1,2,2017-04-10T08:08:10.404723-05:00,2022-04-23T00:53:33.172375-05:00,[],NaN,MADISON,DANE,WI,53705.0,NaN,NaN,[],[]
2,3,2017-04-10T08:21:14.98351-05:00,2022-02-28T19:01:21.95251-06:00,[],NaN,NaN,NaN,NaN,54115.0,NaN,NaN,[],[]
3,4,2017-04-10T08:25:16.115921-05:00,2022-04-23T00:53:33.445243-05:00,[],NaN,MENASHA,WINNEBAGO,WI,54952.0,NaN,NaN,[],[]
4,5,2017-04-10T08:42:59.367602-05:00,2022-04-23T00:53:33.678873-05:00,[],NaN,NEENAH,WINNEBAGO,WI,54956.0,NaN,NaN,[],[]


In [4]:
# Inspect type of time stamp fields
type(clients_raw["Client_CreateStamp"][0])
type(clients_raw["Client_EditStamp"][0])

str

In [5]:
# Make a copy of the data
clients = clients_raw.copy()

In [6]:
# Convert time stamps to date time format (takes a long time, which is why there are separate scripts for each dataset)
clients["Client_CreateStamp"] = pd.to_datetime(clients["Client_CreateStamp"])

In [7]:
# Continue converting time stamps to date time format
clients["Client_EditStamp"] = pd.to_datetime(clients["Client_EditStamp"])

In [8]:
# Some zip codes are not floats
#not_floats = clients_clean["CLIENT_ZIPCODE"].apply(lambda x: not isinstance(x, float))
#clients_clean.shape # (830450, 4)
#not_floats[not_floats].shape #(482703,)
#clients_not_floats = clients_clean[not_floats]

# Some zip codes have dashes in the middle for full zipcode
#long_zipcode = clients_clean["CLIENT_ZIPCODE"].apply(lambda x: len(str(x)) > 7)

# Truncate the zipcode data to 5 chars so that we can better use it for geocoding (some geocoding libraries do not seem to support full zipcodes)
clients["CLIENT_ZIPCODE_TRUNC"] = clients["ClientAddressus_ClientAddressus_zip"].apply(lambda x: str(x)[0:5])

In [9]:
# Get subset of fields
clients_clean = clients[["Client_Id", "Client_CreateStamp", "Client_EditStamp", "CLIENT_ZIPCODE_TRUNC"]]

# Rename columns
clients_clean.rename(
    columns= {
    "Client_Id": "CLIENT_ID", 
    "Client_CreateStamp": "CLIENT_CREATESTAMP",
    "Client_EditStamp": "CLIENT_EDITSTAMP"
    },
    inplace=True
    )

clients_clean.head()

/var/folders/bx/8_p36bpd14nckr66rq2kk6kc0000gn/T/ipykernel_18142/1197042023.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clients_clean.rename(


,CLIENT_ID,CLIENT_CREATESTAMP,CLIENT_EDITSTAMP,CLIENT_ZIPCODE_TRUNC
0,1,2017-04-07 15:34:04.872169-05:00,2022-04-23 00:53:31.812322-05:00,54935
1,2,2017-04-10 08:08:10.404723-05:00,2022-04-23 00:53:33.172375-05:00,53705
2,3,2017-04-10 08:21:14.983510-05:00,2022-02-28 19:01:21.952510-06:00,54115
3,4,2017-04-10 08:25:16.115921-05:00,2022-04-23 00:53:33.445243-05:00,54952
4,5,2017-04-10 08:42:59.367602-05:00,2022-04-23 00:53:33.678873-05:00,54956


In [12]:
# Save output to csv in a `clean/` directory
# Specify index=False so that columns match DB
# Specify header=False to match snowflake input requirements
clients_clean.to_csv("../data/uwwi_dataset_clients_cleaned.csv", index=False, header=False)

In [108]:
# For testing DB upload
#clients_clean.head().to_csv("client_test.csv", index=False, header=False)